### 최종 프로젝트 제출 _ 남정재
#### Titanic Dataset 머신러닝 알고리즘 적용하기

In [85]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [86]:
titanic_df = pd.read_csv("./data/titanic.csv")

In [87]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [88]:
titanic_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [89]:
titanic_df.info()
# Age, Cabin, Embarked 칼럼에 대하여 NaN데이터가 있는 것을 확인할 수 있었다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [90]:
titanic_df = titanic_df.drop(['PassengerId','Name','Ticket'], axis=1)

* 현재 남은 데이터셋에서 Column List
    1. Pclass
    2. Sex
    3. Age
    4. SibSp
    5. Parch
    6. Fare
    7. Cabin
    8. Embarked

In [91]:
titanic_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [92]:
# Pclass : 승객 등급
# 1, 2, 3등급의 데이터인 Pclass는 범주형 데이터이므로
# one-hot-encoding을 pd.get_dummies() 메서드로 인코딩한다.
pcclass_titanic_dummies = pd.get_dummies(titanic_df['Pclass'])

pcclass_titanic_dummies.columns = ['1','2','3']

titanic_df.drop(['Pclass'], axis=1, inplace=True)

titanic_df = titanic_df.join(pcclass_titanic_dummies)

In [93]:
# Sex : 성별
# 성별 데이터인 Sex는 범주형 데이터이므로
# one-hot-encoding을 pd.get_dummies() 메서드로 인코딩한다.
sex_titanic_dummies = pd.get_dummies(titanic_df['Sex'])

sex_titanic_dummies.columns = ['Female', 'Male']

titanic_df.drop(['Sex'], axis=1, inplace=True)

titanic_df = titanic_df.join(sex_titanic_dummies)

In [94]:
# Age : 나이
# 나이는 연속형 데이터이므로, 큰 처리가 필요없다.
# 하지만 일부 NAN 데이터가 있으니 이를 평균값으로 채우기로 하였다.
titanic_df["Age"].fillna(titanic_df["Age"].mean(), inplace=True)

In [95]:
# SibSp : 탑승한 형제자매의 수
# Panch : 탑승한 부모님의 수
# Fare : 탑승료
# SibSp와 Panch와 Fare는 NaN데이터도 없고 별개의 이상이 없으므로 그대로 두었다.

In [96]:
# Cabin : 객실
# NaN데이터가 존재하여서 이에 대한 결측값들은 모두 버렸다.
titanic_df = titanic_df.drop(['Cabin'], axis=1)

In [97]:
# Embarked : 탑승 항구
# 탑승 항구는 C, Q, S로 나누어졌으며
# 각각 C = Cherbourg, Q = Queenstown, S = Southampton를 뜻한다.

# 데이터 확인
titanic_df["Embarked"].value_counts()

# 아래 결과를 보아 S가 가장 많다는 것을 알 수 있었다.

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [98]:
# 하여 결측값이 2개가 존재하였는데 이를 S로 채우도록 하겠다.
titanic_df["Embarked"].fillna('S', inplace=True)

In [99]:
# 범주형 데이터 이므로 위에서 처리한 것과 같이 데이터를 전처리해준다.
embarked_titanic_dummies = pd.get_dummies(titanic_df['Embarked'])

embarked_titanic_dummies.columns = ['S', 'C', 'Q']

titanic_df.drop(['Embarked'], axis=1, inplace=True)

titanic_df = titanic_df.join(embarked_titanic_dummies)

In [100]:
# 데이터 나누기
# (정보, 생존여부) 이러한 형태를 위해 데이터를 나눠준다.

X_titanic = titanic_df.drop("Survived", axis=1)
Y_titanic = titanic_df["Survived"]

In [101]:
# 머신러닝 알고리즘 적용

# Logistic Regression

logreg = LogisticRegression()

logreg.fit(X_titanic, Y_titanic)

logreg.score(X_titanic, Y_titanic)

C:\Users\user\anaconda3\envs\nam1\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8058361391694725

In [102]:
# Support Vector Machines

svc = SVC()

svc.fit(X_titanic, Y_titanic)

svc.score(X_titanic, Y_titanic)

0.6868686868686869

In [103]:
# Random Forests

rnd_for = RandomForestClassifier(n_estimators=100)

rnd_for.fit(X_titanic, Y_titanic)

rnd_for.score(X_titanic, Y_titanic)

0.9820426487093153

In [104]:
# K nearest neighbor algorithm

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_titanic, Y_titanic)

knn.score(X_titanic, Y_titanic)

0.835016835016835

#### 결과적으로 랜덤 포레스트가 가장 좋은 결과를 내는 것을 알 수 있었다.
#### 약 0.982의 Score값을 얻을 수 있었다. 즉, 성능은 약 98.2%이다.